In [75]:
import Model
import library


Scratchpad area: update the above segments as required with the model.py and library.py

In [76]:
video = Video("videos/ball1/side.mp4", (255, 255, 255))
print(video.getDimensions())
print(video.getFPS())


(1920, 1080)
59


In [77]:
while True:
    video.incrementFrame()
    cv.imshow("Frame", video.getCurrentFrame())
    key = cv.waitKey(0)
    if key & 0xFF == ord('q'):
        break
    elif key & 0xFF == ord('n'):
        continue
    elif key & 0xFF == ord('m'):
        video.markFirstFrame()


In [78]:
cv.destroyAllWindows()

In [ ]:
video.getPoints() 

[]

In [74]:
newParameters = Parameters(
    blurSqrSize=15,
    dp=1.0,
    minDist=80,
    minRadius=15,
    maxRadius=50,
    param1=120,
    param2=25
)

video.updateParameters(newParameters)
video.getPoints()

TypeError: 'NoneType' object cannot be interpreted as an integer